# Setup

In [1]:
#import shutil
#!shutil.rmtree('satellite.extractor')
#!git clone https://github.com/sebasmos/satellite.extractor.git

# Install dependencies
#%cd satellite.extractor/

#! python setup.py build
#! python setup.py install
#! pip install -r requirements.txt

#%cd ..

# Import functions

In [2]:
import pandas as pd
import glob
import json
import datetime as dt
from urllib.parse import unquote
from bs4 import BeautifulSoup
import os
import shutil
from sentinelhub import SHConfig
import datetime
import numpy as np
import matplotlib.pyplot as plt
import sys
from datetime import timedelta
sys.path.insert(0,'..')
from sentinelhub import MimeType, CRS, BBox, SentinelHubRequest, SentinelHubDownloadClient, DataCollection, bbox_to_dimensions, DownloadRequest
from epiweeks import Week, Year
import epiweeks
from datetime import date
import glob, shutil

In [3]:
!ls

Coordinates  satellite.extractor  satellite_extractor.ipynb


In [4]:
%cd satellite.extractor
!ls

/home/datascience/Data Fusion/experiments/violence_prediction/Dataset/Satellite/satellite.extractor
distributions.egg-info	linux	   README.md	     scripts   src
__init__.py		notebooks  requirements.txt  setup.py


In [5]:
from linux.request_multiple_images_linux import download_multiple_images

In [6]:
!pwd

/home/datascience/Data Fusion/experiments/violence_prediction/Dataset/Satellite/satellite.extractor


In [7]:
!ls

distributions.egg-info	linux	   README.md	     scripts   src
__init__.py		notebooks  requirements.txt  setup.py


# 10 cities

In [8]:
from dotenv import load_dotenv

load_dotenv()
CLIENT_ID = os.getenv('CLIENT_ID')
CLIENT_SECRET = os.getenv('CLIENT_SECRET')

In [9]:
path = '../Coordinates/'

In [10]:
dic = pd.read_csv(path + 'coordinates512_top_10_municipalities.csv')
dic = dict(zip(dic['Municipality'], dic.square))
# Convert the values to lists
dic = {key: [float(x) for x in value.strip('[]').split(', ')] for key, value in dic.items()}

# List of cities to remove (Just to test)
cities_to_remove = ['Soacha', 'Villavicencio', 'Bucaramanga', 'Ibagué', 'Cúcuta']

# Remove the cities
for city in cities_to_remove:
    if city in dic:
        del dic[city]

dic

{'Medellín': [-75.59778584277811,
  6.2216717557504815,
  -75.55174096123991,
  6.26773709683579],
 'Cali': [-76.54259075677969,
  3.4169717559259674,
  -76.4967364534374,
  3.463037096835792],
 'Pasto': [-77.30395468398439,
  1.1905717560644984,
  -77.2581727224403,
  1.236637096835792],
 'Barranquilla': [-74.81967417403632,
  10.940871755450496,
  -74.77305153207891,
  10.986937096835792],
 'Popayán': [-76.62897028622902,
  2.4180717559881755,
  -76.58315702570543,
  2.4641370968357923]}

In [11]:
img_format = "tiff"

# Define the range of years
initial_year = 2016
end_year = 2023
years = list(range(initial_year,end_year))
first_2015_week = 44

start_2015 = Week(initial_year,first_2015_week).startdate()

weeks_2015 = list(range(first_2015_week, 53))

#weeks = list(range(1,53))
weeks = {}
# Iterate through the years and calculate the number of epiweeks
for year in range(initial_year, end_year):
    num_epiweeks = 0
    for week_num in range(1, 54):
        try:
            epiweek = epiweeks.Week(year, week_num)
            num_epiweeks += 1
        except ValueError:
            # This week doesn't exist for this year
            pass
    weeks[year] = list(range(1, num_epiweeks + 1))

root_images = "/data/"
# Get the absolute path of the current working directory
current_directory = os.getcwd()

temporal_path = os.path.abspath(os.path.join(current_directory, root_images[1:]))

if not os.path.isdir(temporal_path):
    os.makedirs(temporal_path)
    print("Creating temporal data folder")

print(f"Number of cities: {len(dic)}")
print(f"Range: {years}")
print(f"Weeks 2015: {weeks_2015}")

Creating temporal data folder
Number of cities: 5
Range: [2016, 2017, 2018, 2019, 2020, 2021, 2022]
Weeks 2015: [44, 45, 46, 47, 48, 49, 50, 51, 52]


In [12]:
def get_folder_ID(root_images, img_format):

    #print(root_images, img_format)

    walker = "." + root_images

    for root, dirs, files in os.walk(walker, topdown=True):
        for name in files:
            path = os.path.join(root, name)
            if "response" in path:

                folder_path = path.replace("/" + "response." + img_format, "")

    return folder_path

In [13]:
def get_request_individual(request_file, img_format):
    with open(request_file, 'r') as req:
        print(request_file)
        json_decode = json.load(req)
        dataFilter = json_decode['request']["payload"]["input"]["data"][0]
        timeRanges = dataFilter["dataFilter"]["timeRange"]
        start = timeRanges["from"].split("T")[0]
        end = timeRanges["to"].split("T")[0]
        time_stamp = start + "_to_" + end # best image calculated on this interval
        name_image = "image_" +   start
        print(name_image)
        data = {
            "start": start,
            "end": end,
            "time_stamp": time_stamp,
            "url": request_file,
            }

        df = pd.DataFrame(data, index =['date'])

        # Update name of files using JSON timestamp filter
        response_img = request_file.split("request.json")[0] + "response.tiff"# + "."+ img_format
        image_path = os.path.join(response_img).replace('response', name_image )
        print("{0} renamed to {1}".format(response_img, image_path))
        shutil.move(response_img, image_path)

        return df

In [14]:
def get_images(coordenates, years, weeks, weeks_2015, img_format, root_images, CLIENT_ID, CLIENT_SECRET):
    # Download data
    folder_path = ""
    for year in years:
        if year == 2015:
            for week in weeks_2015:
                print(f"Year: {year} - week: {week}")
                # Set starting date for given year based on
                start = Week(year, week).startdate()
                # Download individual images
                download_multiple_images(coordenates, start, str(year), CLIENT_ID, CLIENT_SECRET)
                # Obtain ID as the last-obtained response.tiff file, "real-time"
                folder_path = get_folder_ID(root_images, img_format)
                # Rename image based on JSON timestamp filter
                dates = get_request_individual(folder_path+"/request.json", img_format)
                # Clean folders which contain black images
                path_to_blank_ids  = "." + root_images + "/" + str(year) + "/*/*"
                ids = glob.glob(path_to_blank_ids)
                for idx in ids:
                    if "response" in idx:
                        folder_path = idx.replace("/" + "response." + img_format, "")
                        #print(folder_path)
                        shutil.rmtree(folder_path)
                ids = glob.glob(path_to_blank_ids)
                #print("ids available: ", ids)

                #flag =  [idx if ("response" in idx) for idx in ids]
        #elif year ==2016:
        else:
            for week in weeks[year]:
                print(f"Year: {year} - week: {week}")
                # Set starting date for given year based on
                start = Week(year, week).startdate()
                # Download individual images
                download_multiple_images(coordenates, start, str(year), CLIENT_ID, CLIENT_SECRET)
                # Obtain ID as the last-obtained response.tiff file, "real-time"
                folder_path = get_folder_ID(root_images, img_format)
                # Rename image based on JSON timestamp filter
                dates = get_request_individual(folder_path+"/request.json", img_format)
                # Clean folders which contain black images
                path_to_blank_ids  = "." + root_images + "/" + str(year) + "/*/*"
                ids = glob.glob(path_to_blank_ids)
                for idx in ids:
                    if "response" in idx:
                        folder_path = idx.replace("/" + "response." + img_format, "")
                        print(folder_path)
                        shutil.rmtree(folder_path)
                ids = glob.glob(path_to_blank_ids)


In [ ]:
for i in dic:
    print(f"City: {i} - Coordinates: {dic[i]}")

    current_coor = dic[i]

    city_str = "DATASET" + "/" + str(i)

    if not os.path.exists(city_str):
        os.makedirs(city_str)

    # Download images on given range
    get_images(current_coor, years, weeks, weeks_2015, img_format, root_images, CLIENT_ID, CLIENT_SECRET)

    # Move to structured folder in DATASETS
    root_images_store = "." + root_images
    dataset_store = "./" + city_str
    for root, dirs, files in os.walk(root_images_store, topdown=True):
        for name in files:
            path = os.path.join(root, name)
            #print(path)
            if img_format in path and not dataset_store in path:
                shutil.copy(path, dataset_store)

images = glob.glob(dataset_store+"/*")

City: Medellín - Coordinates: [-75.59778584277811, 6.2216717557504815, -75.55174096123991, 6.26773709683579]
Year: 2016 - week: 1
Requested week slot: 01/03/2016 - 01/10/2016 
Ended downloading
./data/2016/53fea6c383643089f87b7edb50ae8340/request.json
image_2016-01-03
./data/2016/53fea6c383643089f87b7edb50ae8340/response.tiff renamed to ./data/2016/53fea6c383643089f87b7edb50ae8340/image_2016-01-03.tiff
Year: 2016 - week: 2
Requested week slot: 01/10/2016 - 01/17/2016 
Ended downloading
./data/2016/7af2ba8f95af426fb18dc60b738fb8aa/request.json
image_2016-01-10
./data/2016/7af2ba8f95af426fb18dc60b738fb8aa/response.tiff renamed to ./data/2016/7af2ba8f95af426fb18dc60b738fb8aa/image_2016-01-10.tiff
Year: 2016 - week: 3
Requested week slot: 01/17/2016 - 01/24/2016 


In [24]:
#shutil.rmtree("DATASET")
#shutil.rmtree("data")

In [ ]:
def plot_image(image, factor=1.0, clip_range = None, **kwargs):
    """
    Utility function for plotting RGB images.
    """
    fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(15, 15))
    if clip_range is not None:
        ax.imshow(np.clip(image * factor, *clip_range), **kwargs)
    else:
        ax.imshow(image * factor, **kwargs)
    ax.set_xticks([])
    ax.set_yticks([])


In [ ]:
images

In [ ]:
from skimage import io
import numpy as np
import cv2

ncols = 4
nrows = 3

aspect_ratio = 1#coordinates_size[0] / coordinates_size[1]
subplot_kw = {'xticks': [], 'yticks': [], 'frame_on': False}

fig, axs = plt.subplots(ncols=ncols, nrows=nrows, figsize=(5 * ncols * aspect_ratio, 5 * nrows),
                            subplot_kw=subplot_kw)

for idx, path in enumerate(images):
        ax = axs[idx // ncols][idx % ncols]
        image = io.imread(path)
        image = np.array(image)
        image = image[:,:,1:4]
        ax.imshow(np.clip(image * 2.5/255, 0, 1))

plt.tight_layout()



# Move to structured folder in DATASETS
root_images_store = "." + root_images
dataset_store = "./" + city_str
for root, dirs, files in os.walk(root_images_store, topdown=True):
        for name in files:
          path = os.path.join(root, name)
          #print(path)
          if img_format in path and not dataset_store in path:
              shutil.copy(path, dataset_store)

images = glob.glob(dataset_store+"/*")